In [131]:
#importa a biblioteca pandas
import pandas as pd

#importa o módulo de conexão com o MySQL
import mysql.connector

#importa o matplotlib (plotagem de gráficos)
import matplotlib.pyplot as plt

#importa a funçao train_test_split, para separar o dataset em dados de teste e dados de treinamento
from sklearn.model_selection import train_test_split

#importa o algoritmo KNN (K nearest neighbors) e ferramentas de medição do resultado (metrics)
from sklearn import neighbors, metrics

#importar o SVM (Support Vector Machine)
from sklearn import svm

#importa o Naive Bayes
from sklearn.naive_bayes import GaussianNB

#importa Regressão Logística
from sklearn.linear_model import LogisticRegression

#importa algoritmo de aplicação de Random Forest
from sklearn.ensemble import RandomForestClassifier

#importa a matriz de confusão e o relatório de classificação do scikit
from sklearn.metrics import confusion_matrix, classification_report

#importa o cálculo de médias do numpy
from numpy import mean

#importa o método cross_val_score para a aferição de resultados após aplicação de validação cruzada
from sklearn.model_selection import cross_val_score

#importa o stats para aplicação de cálculos estatísticos
import scipy.stats as stats

#importa contador do Python, para medição dos resultados de classes de variável
from collections import Counter

#importa SMOTE da biblioteca imblearn, para aplicação de Oversampling na base de treino
from imblearn.over_sampling import SMOTE

#importa RandomUnderSampler da biblioteca imblearn, para aplicação de Undersampling na base de treino
from imblearn.under_sampling import RandomUnderSampler

#importa o RepeatedStratifiedKFold para aplicação repetida de particionamentos de validação cruzada na base de dados
from sklearn.model_selection import RepeatedStratifiedKFold

#importa Pipeline da imblearn, para aplicação das estratégias de Resampling
from imblearn.pipeline import Pipeline

In [132]:
db = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd="mydbwork",
    db = "puc")


In [133]:
mycursor = db.cursor()

mycursor.execute("""SELECT TXT_CRGO, VL_REND_LQDO, idade, COD_GRAU_INST, COD_ETDO_CVIL FROM cadastro""")
dados = mycursor.fetchall()


In [134]:
df = pd.DataFrame(dados, columns=['Profissao','Renda','Idade', 'Escolaridade', 'Estado_Civil'])

In [77]:
df.isnull().values.any()

False

In [78]:
df.isnull().sum()

Profissao       0
Renda           0
Idade           0
Escolaridade    0
Estado_Civil    0
dtype: int64

In [105]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33243 entries, 0 to 33242
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Profissao     33243 non-null  object
 1   Renda         33243 non-null  object
 2   Idade         33243 non-null  int64 
 3   Escolaridade  33243 non-null  int64 
 4   Estado_Civil  33243 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.3+ MB


In [69]:
#dados do tipo 'object' são tratados como texto pelo pandas. Código de erro.
df['Renda'] = df['Renda'].astype('float')

ValueError: could not convert string to float: '3203,16'

In [70]:
#impossibilidade de transformar renda em float, pela existência de vírgulas no lugar de pontos

In [71]:
df['Renda'] = df['Renda'].str.replace(',','.')

In [72]:
df['Renda'] = df['Renda'].astype('float')

ValueError: could not convert string to float: 

In [73]:
#Ainda havia registros do tipo  "" na base. Realizei os ajustes, preechendo espaços vazios com 0, para posterior adoção
#de estratégia de atribuição de valor a estes campos

In [80]:
df['Renda'] = df['Renda'].str.replace('','0')

In [81]:
df['Renda'] = df['Renda'].astype('float')

In [82]:
df[df["Renda"]==0]

,Profissao,Renda,Idade,Escolaridade,Estado_Civil
16,ESTUDANTE,0.0,48,5,1
45,MEDICA,0.0,55,5,1
90,,0.0,66,3,2
106,,0.0,46,6,1
144,,0.0,77,2,2
...,...,...,...,...,...
33145,,0.0,33,3,7
33173,,0.0,40,5,1
33180,,0.0,33,3,1
33212,,0.0,60,5,3


In [ ]:
#2445 campos de renda com informação de renda 0. A estratégia de ajuste, com atribuição de valores será adotada em momento
#oportuno

In [ ]:
#presença de "" no campo de Profissão

In [85]:
mask = df["Profissao"] == "" 
df[mask].value_counts()

Profissao  Renda       Idade  Escolaridade  Estado_Civil
           0.0         23     9             1               64
                       24     9             1               63
                       22     9             1               59
                       25     9             1               57
                       26     9             1               53
                                                            ..
           10102070.0  25     2             3                1
           10105000.0  21     2             1                1
                              3             1                1
           10107030.0  29     3             3                1
           0.0         1      1             1                1
Length: 1328, dtype: int64

In [ ]:
#1328 registros omissos de profissão

In [135]:
#idades registradas como 0
mask = df["Idade"] ==0
df[mask]

,Profissao,Renda,Idade,Escolaridade,Estado_Civil
4072,TEC SEGURANCA DO TRABALHO,1066.6,0,3,1
4073,VENDEDORA,1689.8,0,3,1
4074,A,2783.09,0,3,1
4075,MOCO DE MAQUINAS,8051.93,0,3,3
4076,,669.57,0,9,1
...,...,...,...,...,...
5016,VENDEDOR AUTONOMO,1587.8,0,5,3
5017,ESCRIVA DE POLICIA FEDERAL,14421.8,0,5,7
5018,TECNICO DE ENFERMAGEM,5154.81,0,3,3
5019,ATENDENTE COMERCIAL,2232.32,0,3,1
